Imports

In [ ]:
import os
import numpy as np
from PIL import Image

### Helper functions

Function to normalize a 2d array

In [ ]:
def GetMaxValue(image_array):
    x, y = image_array.shape
    max_value = 0
    for i in range(x):
        for j in range(y):
            value = image_array[i][j]
            if value > max_value:
                max_value = value
    return max_value

def NormalizeArray(image_array):
    x, y = image_array.shape
    max_value = GetMaxValue(image_array)
    normalized_image_array = np.copy(image_array)
    for i in range(x):
        for j in range(y):
            old_value = image_array[i][j]
            new_value = round(old_value / max_value * 255)
            normalized_image_array[i][j] = new_value
    return normalized_image_array

Function to convert a greyscale image to 2D array

In [ ]:
def ImageToArray(image):
    image_array = np.array(image.getdata(), dtype=np.uint8)
    image_array = image_array.reshape(image.size)
    return image_array

Function to convert a 2d array to an image

In [ ]:
def ArrayToImage(image_array):
    x, y = image_array.shape
    image = Image.new("L", (x, y))
    for i in range(x):
        for j in range(y):
            value = int(image_array[i][j])
            image.putpixel((i, j), value)
    return image

### Main functions

CalculateIntegral function

In [ ]:
def CalculateIntegral(image_array):
    x, y = image_array.shape
    row_sums = np.zeros((x+1, y+1), dtype=np.uint32)
    integral_image = np.zeros((x+1, y+1), dtype=np.uint32)
    for i in range(x):
        for j in range(y):
            r = i+1
            c = j+1
            row_sums[r][c] = row_sums[r][c-1] + image_array[i][j]
            integral_image[r][c] = integral_image[r-1][c] + row_sums[r][c]
    integral_image = np.delete(integral_image, 0, axis=0)
    integral_image = np.delete(integral_image, 0, axis=1)
    return integral_image

CalculateLocalSum function

In [ ]:
def CalculateLocalSum(integral_image, p0, p1):
    x0, y0 = p0
    x1, y1 = p1
    sum_x = integral_image[x0][y0]
    sum_y = integral_image[x0][y1]
    sum_z = integral_image[x1][y0]
    sum_k = integral_image[x1][y1]
    local_sum = sum_k + sum_x - sum_z - sum_y
    return local_sum

### Test

Test 1

In [ ]:
image_array = np.full((200, 200), 100, dtype=np.uint8)
image = ArrayToImage(image_array)

In [ ]:
integral_image = CalculateIntegral(image_array)
ArrayToImage(NormalizeArray(integral_image))

In [ ]:
CalculateLocalSum(integral_image, (20, 20), (30, 30))

Test 2

In [ ]:
current_directory = os.getcwd()
image_path = current_directory + R"\images\f2.png"
image = Image.open(image_path)
image = image.convert("L")
image_array = ImageToArray(image)